<a href="https://colab.research.google.com/github/kridtapon/Squeeze-Pop-Strategy/blob/main/Squeeze_%26_Pop_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20, std_dev=2):
    df['SMA'] = df['Close'].rolling(window=period).mean()
    df['STD'] = df['Close'].rolling(window=period).std()
    df['Upper_BB'] = df['SMA'] + (df['STD'] * std_dev)
    df['Lower_BB'] = df['SMA'] - (df['STD'] * std_dev)
    df['BB_Width'] = df['Upper_BB'] - df['Lower_BB']  # Bollinger Band Width
    return df

# Define the stock symbol and time period
symbol = 'AXON'  # Example: META (Facebook)
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Bollinger Bands
df = calculate_bollinger_bands(df)

# Define squeeze condition (BB width lowest in last 50 periods)
df['Squeeze'] = df['BB_Width'] == df['BB_Width'].rolling(50).min()

# Define breakout condition
# Entry when full candle closes above upper BB
df['Entry'] = df['Squeeze']

# Exit if price moves back inside the bands
df['Exit'] = df['Close'] < df['Lower_BB']

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           434945.838777
Total Return [%]                       334.945839
Benchmark Return [%]                   676.280076
Max Gross Exposure [%]                      100.0
Total Fees Paid                       2945.306099
Max Drawdown [%]                         40.07101
Max Drawdown Duration                       389.0
Total Trades                                   12
Total Closed Trades                            11
Total Open Trades                               1
Open Trade PnL                      216361.710633
Win Rate [%]                            45.454545
Best Trade [%]                          74.016407
Worst Trade [%]                        -22.326183
Avg Winning Trade [%]                   34.656419
Avg Losing Trade [%]                      -9.8703
